# Scikit-Learn Data Preprocessor
## Using TITANIC_VIEW from DWC. This view has 861 records

## Install fedml_gcp package

In [ ]:
pip install fedml_gcp

## Import Libraries

In [ ]:
from fedml_gcp import DwcGCP
import numpy as np
import pandas as pd

## Create DwcGCP Instance to access class methods and train model

It is expected that the bucket name passed here already exists in Cloud Storage.

In [ ]:
dwc = DwcGCP(project_name='example-project',
                 bucket_name='<bucket-name>')

### Create tar bundle of script folder so GCP can use it for training

Before running this cell, please ensure that the script package has all the necessary files for a training job.

In [ ]:
dwc.make_tar_bundle('DataPreprocessor.tar.gz', 'DataPreprocessor', 'datapreprocessor/train/DataPreprocessor.tar.gz')


### Create tar bundle of predictor script folder so GCP can use it for inferencing

Before running this cell, please ensure that the predictor package has all the necessary files for a training job.

In [ ]:
dwc.make_tar_bundle('DataPreprocessorPredictor.tar.gz', 'DataPreprocessorPredictor', 'datapreprocessor/prediction/DataPreprocessorPredictor.tar.gz')


### Train Model

GCP takes in training inputs that are specific to the training job and the environment needed.

In the training inputs, we are the python module. This is the module that your script package is named, and it references the task.py file inside the script package.

We are also passing args which hold the table name to get data from. Before running the following cell, you should have a config.json file in the script package with the specified values to allow you to access to DWC.

You should also have the follow view `TITANIC_VIEW` created in your DWC. To gather this data, please refer to https://www.kaggle.com/c/titanic/data and download the train.csv file.

In [ ]:
training_inputs = {
    'scaleTier': 'BASIC',
    'packageUris': ['gs://<bucket-name>/datapreprocessor/train/DataPreprocessor.tar.gz'],
    'pythonModule': 'trainer.task',
    'args': ['--table_name', 'TITANIC_VIEW', '--bucket_name', '<bucket-name>'],
    'region': 'us-east1',
    'jobDir': 'gs://<bucket-name>',
    'runtimeVersion': '2.5',
    'pythonVersion': '3.7',
    'scheduling': {'maxWaitTime': '3600s', 'maxRunningTime': '7200s'}
}

In [ ]:
dwc.train_model('preprocessor_final_train1', training_inputs)

### Deploy model

In [ ]:
dwc.deploy(model_name='<example-model>',  
           model_location='/datapreprocessor/model', version='v1', region='us-east1', 
           prediction_location='datapreprocessor/prediction/', custom_predict='DataPreprocessorPredictor.tar.gz', module_name='predictor.MyPredictor', 
           onlinePredictionLogging=True, onlinePredictionConsoleLogging=True)
